# Dashcam Models (Colab version)

- Use GPU runtime (L4 or T4 recommended)

## Imports

In [ ]:
# Ensure latest version of tensorflow
# Note: Colab is sometimes a few versions behind documentation
!pip install tensorflow --upgrade
!pip install tf-keras --upgrade
!pip install tf-keras-vis

In [ ]:
# Standard data imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Files
from google.colab import drive
import zipfile
import os
import shutil

# Image
from PIL import Image

# ML modeling imports
import tensorflow as tf
from tf_keras_vis.saliency import Saliency
from tf_keras_vis.utils.scores import BinaryScore

## Data

In [ ]:
DATA_PATH = "drive/MyDrive/dashcam-ai" # Path to data zip file (change as needed)
DATA_FOLDER_NAME = "slowroads dataset" # Name of folder generated from zip extraction
TEST_FOLDER_NAME = "test set" # Same as above for test set

In [ ]:
# Mount drive
drive.mount('/content/drive')

# Unzip data
with zipfile.ZipFile(f"{DATA_PATH}/{DATA_FOLDER_NAME}.zip", 'r') as f:
    f.extractall("/content")

train = tf.data.Dataset.load(f"/content/{DATA_FOLDER_NAME}/train")
val = tf.data.Dataset.load(f"/content/{DATA_FOLDER_NAME}/val")

# Unzip test datasets
with zipfile.ZipFile(f"{DATA_PATH}/{TEST_FOLDER_NAME}.zip", 'r') as f:
    f.extractall("/content")


easy = tf.data.Dataset.load(f"/content/{TEST_FOLDER_NAME}/easy")
hard = tf.data.Dataset.load(f"/content/{TEST_FOLDER_NAME}/hard")
guard = tf.data.Dataset.load(f"/content/{TEST_FOLDER_NAME}/metal guard")
fence = tf.data.Dataset.load(f"/content/{TEST_FOLDER_NAME}/wood fence")

In [ ]:
# Manually inspect dataset
ds = val
sample_index = 22

# Show selected sample
curr_index = 0
for x, y in ds:
    # Skip unselected samples
    if curr_index != sample_index:
        curr_index += 1

        continue

    # Show and stop iterating
    plt.imshow(x)
    break

## Models

### Utilities
- Random seeds
- Toy training data (single-sample "dataset" for fast debug)
- Saliency map
- Parameter counter
- Basic trainer

In [ ]:
SEEDS = [3141, 2718, 1414, 2024]

In [ ]:
toy_train = None

for x, y in train:
    toy_train = tf.data.Dataset.from_tensor_slices(([x], [y]))
    break

In [ ]:
def get_saliency(model, x):
    # Create saliency object
    saliency = Saliency(model, clone=False)
    score = BinaryScore([0])

    # Compute and noramlize map
    map = saliency(score, x, smooth_samples=20, smooth_noise=0.2)
    map = (map - map.min())/(map.max() - map.min())

    return map

In [ ]:
def count_params(model):
    params = 0

    for layer in model.layers:
        for variable in layer.trainable_variables:
            shape = variable.shape
            params += np.prod(shape)

    return params

In [ ]:
def get_acc(model, batched_ds):
    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )

    loss, acc = model.evaluate(
        x=batched_ds,
        verbose=0
    )

    return acc

In [ ]:
class GetBestWeights(tf.keras.callbacks.Callback):
    def __init__(self, metric="val_loss", max_=False):
        self.metric = metric
        self.max = max_

        self.best = float("-inf") if max_ else float("inf")
        self.best_weights = None

    def on_epoch_end(self, epoch, logs=None):
        # Get metric
        metric = logs[self.metric]

        # Do not update if not improved
        if (self.max) and (metric < self.best):
            return

        if (not self.max) and (metric > self.best):
            return

        # Update if improved
        self.best = metric
        self.best_weights = self.model.get_weights()

In [ ]:
def train_until_best(model, save_path, train_batched=None, val_batched=None, verbose=1):
    # Compile model
    optimizer = tf.keras.optimizers.Adam(
        learning_rate=1e-3
    )

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=2,
    )

    get_best_weights = GetBestWeights("val_loss")

    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(),
        optimizer=optimizer,
        metrics=[tf.keras.metrics.BinaryAccuracy()],
    )

    # Default data
    if not train_batched:
        train_batched = train.batch(16)

    if not val_batched:
        val_batched = val.batch(16)

    # Train on high learning rate
    model.fit(
        x=train_batched,
        validation_data=val_batched,
        callbacks=[early_stopping, get_best_weights],
        epochs=10,
        verbose=verbose
    )

    # Recover to best weights
    best_weights = get_best_weights.best_weights
    model.set_weights(best_weights)

    # Train on lower learning rate
    optimizer.learning_rate = 1e-4

    model.fit(
        x=train_batched,
        validation_data=val_batched,
        callbacks=[early_stopping, get_best_weights],
        epochs=10,
        verbose=verbose
    )

    # Save best weights
    best_weights = get_best_weights.best_weights
    model.set_weights(best_weights)
    model.save_weights(save_path)

### (Relatively) Large Baseline Model

In [ ]:
large_model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(
        shape=(382, 512, 3)
    ),
    tf.keras.layers.Conv2D(
        filters=4,
        kernel_size=(8, 8),
    ),
    tf.keras.layers.AveragePooling2D(
        pool_size=(2, 2)
    ),
    tf.keras.layers.Conv2D(
        filters=8,
        kernel_size=(8, 8),
    ),
    tf.keras.layers.AveragePooling2D(
        pool_size=(4, 4)
    ),
    tf.keras.layers.Conv2D(
        filters=16,
        kernel_size=(8, 8),
    ),
    tf.keras.layers.MaxPool2D(
        pool_size=(4, 4)
    ),
    tf.keras.layers.Conv2D(
        filters=32,
        kernel_size=(4, 4),
    ),
    tf.keras.layers.MaxPool2D(
        pool_size=(4, 4)
    ),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Activation("sigmoid")
])

#### Train and Save Weights

#### Load Saved Weights

In [ ]:
large_model.load_weights(f"{DATA_PATH}/large.weights.h5")

### Smaller Models

In [ ]:
def layer_skipped_model(layer_mask):
    # layer_mask must be a boolean list of length 4
    assert len(layer_mask) == 4
    assert sum([type(i) == bool for i in layer_mask]) == 4

    # Define layers
    layer_1 = [
        tf.keras.layers.Conv2D(
            filters=4,
            kernel_size=(8, 8),
        ),
        tf.keras.layers.AveragePooling2D(
            pool_size=(2, 2)
        )
    ]

    layer_2 = [
        tf.keras.layers.Conv2D(
            filters=8,
            kernel_size=(8, 8),
        ),
        tf.keras.layers.AveragePooling2D(
            pool_size=(4, 4)
        )
    ]

    layer_3 = [
        tf.keras.layers.Conv2D(
            filters=16,
            kernel_size=(8, 8),
        ),
        tf.keras.layers.MaxPool2D(
            pool_size=(4, 4)
        )
    ]

    layer_4 = [
        tf.keras.layers.Conv2D(
            filters=32,
            kernel_size=(4, 4),
        ),
        tf.keras.layers.MaxPool2D(
            pool_size=(4, 4)
        )
    ]

    layers = [layer_1, layer_2, layer_3, layer_4]

    # Begin sequence
    sequence = [
        tf.keras.layers.InputLayer(
            shape=(382, 512, 3)
        )
    ]

    # Add layers with skipping
    skipped_pool_multiplier = 1
    for i, skipped in enumerate(layer_mask):
        # Get layer information
        layer = layers[i]
        pool_size = layer[1].pool_size[0]

        # Skip layer and record amount of pool balancing needed
        if skipped:
            skipped_pool_multiplier *= pool_size
            continue

        # Compute new pool size
        new_pool_size = pool_size * skipped_pool_multiplier

        # Reset skipped pool multiplier (new pool size compensates for skipped)
        skipped_pool_multiplier = 1

        # Update layer pool size
        layer[1].pool_size = (new_pool_size, new_pool_size)
        layer[1].strides = layer[1].pool_size # consistent with default tf

        # Add conv and pool layers to sequence
        sequence += layer

    # Add extra pooling if needed
    if skipped_pool_multiplier != 1:
        extra_pool = tf.keras.layers.MaxPool2D(
            pool_size=(skipped_pool_multiplier, skipped_pool_multiplier)
        )

        sequence.append(extra_pool)

    # Complete sequence
    sequence += [
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(32),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1),
        tf.keras.layers.Activation("sigmoid")
    ]

    # Create model from sequence
    return tf.keras.Sequential(sequence)

## Scaling tests
- Plot val accuracy vs param count
- Examine target task proficiency vs param count

In [ ]:
param_counts = []
val_accs = []

### Baseline

In [ ]:
param_counts.append(count_params(large_model))
val_accs.append(get_acc(large_model, val.batch(16)))

### Smaller

In [ ]:
for i in range(1, 16):
    skip_mask = [c == "1" for c in bin(i)[2:].zfill(4)]

    acc_sum = 0

    print(f"Training model {i}/15:")

    for j, seed in enumerate(SEEDS):
        tf.random.set_seed(seed)

        model = layer_skipped_model(skip_mask)

        print(f"Training Seed {j+1}/{len(SEEDS)} ... ", end="")
        train_until_best(model, f"scaled{i}_{j}.weights.h5", verbose=0)
        print("Done!")

        acc_sum += get_acc(model, val.batch(16))

    param_counts.append(count_params(model))
    val_accs.append(acc_sum/len(SEEDS))

    print(f"Stats - {param_counts[-1]} params, {val_accs[-1]:.5f} val acc")

In [ ]:
all_file_names = []

for i in range(1, 16):
    for j in range(4):
        all_file_names.append(f"scaled{i}_{j}.weights.h5")

In [ ]:
os.mkdir("scaled models")

for file_name in all_file_names:
    os.rename(file_name, f"scaled models/{file_name}")

In [ ]:
shutil.make_archive("scaled models", "zip", "scaled models")

## Analysis

In [ ]:
# Unzip small models
with zipfile.ZipFile(f"{DATA_PATH}/scaled models.zip", 'r') as f:
    f.extractall("/content")

In [ ]:
def build_model(weights_file_name):
    i = int(weights_file_name[6:].split("_")[0])
    skip_mask = [c == "1" for c in bin(i)[2:].zfill(4)]

    model = layer_skipped_model(skip_mask)

    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()],
    )

    model.load_weights(weights_file_name)

    return model

In [ ]:
models = {}

for i in range(1, 16):
    temp = []

    for j in range(4):
        temp.append(build_model(f"scaled{i}_{j}.weights.h5"))

    params = count_params(temp[-1])
    models[params] = temp

In [ ]:
param_counts = sorted(list(models.keys()))

In [ ]:
def get_sample(n, ds):
    i = 0

    for img, y in ds:
        if i != n:
            i += 1
        else:
            break

    return img

### Saliency Maps

In [ ]:
fence_crash = get_sample(5, fence)

plt.imshow(fence_crash)
plt.show()

In [ ]:
fence_drive = get_sample(10, fence)

plt.imshow(fence_drive)
plt.show()

In [ ]:
def group_saliency(params, x):
    maps = [get_saliency(model, x) for model in models[params]]

    summed = np.sum(np.concatenate(maps, axis=0), axis=0)

    return np.min([summed, np.ones_like(summed)], axis=0)

In [ ]:
for param_count in param_counts:
    plt.axis('off')
    plt.imshow(fence_drive)
    plt.imshow(group_saliency(param_count, fence_drive), cmap="jet", alpha=0.3)
    plt.title(f"{param_count} params")
    plt.show()

In [ ]:
for param_count in param_counts:
    plt.axis('off')
    plt.imshow(fence_crash)
    plt.imshow(group_saliency(param_count, fence_crash), cmap="jet", alpha=0.3)
    plt.title(f"{param_count} params")
    plt.show()

In [ ]:
guard_crash = get_sample(20, guard)

plt.imshow(guard_crash)
plt.show()

In [ ]:
guard_drive = get_sample(5, guard)

plt.imshow(guard_drive)
plt.show()

In [ ]:
for param_count in param_counts:
    plt.axis('off')
    plt.imshow(guard_drive)
    plt.imshow(group_saliency(param_count, guard_drive), cmap="jet", alpha=0.3)
    plt.title(f"{param_count} params")
    plt.show()

In [ ]:
for param_count in param_counts:
    plt.axis('off')
    plt.imshow(guard_crash)
    plt.imshow(group_saliency(param_count, guard_crash), cmap="jet", alpha=0.3)
    plt.title(f"{param_count} params")
    plt.show()

In [ ]:
hard_crash = get_sample(5, hard)

plt.imshow(hard_crash)
plt.show()

In [ ]:
hard_drive = get_sample(20, hard)

plt.imshow(hard_drive)
plt.show()

In [ ]:
for param_count in param_counts:
    plt.axis('off')
    plt.imshow(hard_crash)
    plt.imshow(group_saliency(param_count, hard_crash), cmap="jet", alpha=0.3)
    plt.title(f"{param_count} params")
    plt.show()

In [ ]:
for param_count in param_counts:
    plt.axis('off')
    plt.imshow(hard_drive)
    plt.imshow(group_saliency(param_count, hard_drive), cmap="jet", alpha=0.3)
    plt.title(f"{param_count} params")
    plt.show()

### Scaling Curves

In [ ]:
def scaling_curve(test_set):
    y_min = []
    y = []
    y_max = []

    for param_count in tqdm(param_counts):
        accs = []

        for model in models[param_count]:
            loss, acc = model.evaluate(
                x=test_set.batch(16),
                verbose=0
            )

            accs.append(acc)

        accs = np.array(accs)

        y_min.append(accs.min())
        y.append(accs.mean())
        y_max.append(accs.max())

    return {"min": y_min, "mean": y, "max": y_max}

In [ ]:
val_curve = scaling_curve(val)

In [ ]:
fence_curve = scaling_curve(fence)

In [ ]:
# Fence
plt.plot(param_counts, fence_curve["mean"], label="Wood Fence")
plt.fill_between(param_counts, fence_curve["min"], fence_curve["max"], alpha=0.3)

plt.plot(param_counts, val_curve["mean"], linestyle="--", alpha=0.5, label="Validation")
plt.fill_between(param_counts, val_curve["min"], val_curve["max"], alpha=0.2)

plt.legend(loc="lower right")
plt.xlabel("Parameters")
plt.ylabel("Accuracy")
plt.ylim(0, 1)

plt.show()

In [ ]:
easy_curve = scaling_curve(easy)

In [ ]:
# Easy
plt.plot(param_counts, easy_curve["mean"], label="Easy")
plt.fill_between(param_counts, easy_curve["min"], easy_curve["max"], alpha=0.3)

plt.plot(param_counts, val_curve["mean"], linestyle="--", alpha=0.5, label="Validation")
plt.fill_between(param_counts, val_curve["min"], val_curve["max"], alpha=0.2)

plt.legend(loc="lower right")
plt.xlabel("Parameters")
plt.ylabel("Accuracy")
plt.ylim(0, 1)

plt.show()

In [ ]:
hard_curve = scaling_curve(hard)

In [ ]:
# Hard
plt.plot(param_counts, hard_curve["mean"], label="Hard")
plt.fill_between(param_counts, hard_curve["min"], hard_curve["max"], alpha=0.3)

plt.plot(param_counts, val_curve["mean"], linestyle="--", alpha=0.5, label="Validation")
plt.fill_between(param_counts, val_curve["min"], val_curve["max"], alpha=0.2)

plt.legend(loc="lower right")
plt.xlabel("Parameters")
plt.ylabel("Accuracy")
plt.ylim(0, 1)

plt.show()

In [ ]:
guard_curve = scaling_curve(guard)

In [ ]:
# Guard
plt.plot(param_counts, guard_curve["mean"], label="Metal Guard")
plt.fill_between(param_counts, guard_curve["min"], guard_curve["max"], alpha=0.3)

plt.plot(param_counts, val_curve["mean"], linestyle="--", alpha=0.5, label="Validation")
plt.fill_between(param_counts, val_curve["min"], val_curve["max"], alpha=0.2)

plt.legend(loc="lower right")
plt.xlabel("Parameters")
plt.ylabel("Accuracy")
plt.ylim(0, 1)

plt.show()

In [ ]:
# Val
plt.rcParams.update({'font.size': 14})

plt.plot(param_counts, val_curve["mean"], c="C1")
plt.fill_between(param_counts, val_curve["min"], val_curve["max"], alpha=0.3, color="C1")
plt.xlabel("Parameters")
plt.ylabel("Accuracy")
plt.ylim(0, 1)
plt.show()

In [ ]:
# Guard v Fence Comparison
plt.plot(param_counts, guard_curve["mean"], label="Metal Guard", c="C2")
plt.fill_between(param_counts, guard_curve["min"], guard_curve["max"], alpha=0.3, color="C2")

plt.plot(param_counts, fence_curve["mean"], label="Wood Fence", c="C5")
plt.fill_between(param_counts, fence_curve["min"], fence_curve["max"], alpha=0.3, color="C5")

plt.plot(param_counts, val_curve["mean"], linestyle="--", alpha=0.5, label="Validation", c="C1")
plt.fill_between(param_counts, val_curve["min"], val_curve["max"], alpha=0.2, color="C1")

plt.legend(loc="lower right")
plt.xlabel("Parameters")
# plt.ylabel("Accuracy")
plt.ylim(0, 1)

plt.show()

In [ ]:
# Easy v Hard Comparison
plt.plot(param_counts, easy_curve["mean"], label="Easy", c="C3")
plt.fill_between(param_counts, easy_curve["min"], easy_curve["max"], alpha=0.3, color="C3")

plt.plot(param_counts, hard_curve["mean"], label="Hard", c="C4")
plt.fill_between(param_counts, hard_curve["min"], hard_curve["max"], alpha=0.3, color="C4")

plt.plot(param_counts, val_curve["mean"], linestyle="--", alpha=0.5, label="Validation", c="C1")
plt.fill_between(param_counts, val_curve["min"], val_curve["max"], alpha=0.2, color="C1")

plt.legend(loc="lower right")
plt.xlabel("Parameters")
# plt.ylabel("Accuracy")
plt.ylim(0, 1)

plt.show()

#### Extras

Below are shortcuts to set the raw results I obtained

In [ ]:
param_counts = sorted([14173, 13165, 4429, 19317, 8021, 9061, 1605, 20089, 12889, 11881, 3145, 17521, 6225, 7777, 833])

In [ ]:
val_curve = {'min': [0.5249999761581421,
  0.4399999976158142,
  0.5699999928474426,
  0.7749999761581421,
  0.625,
  0.44999998807907104,
  0.7724999785423279,
  0.7124999761581421,
  0.8575000166893005,
  0.8475000262260437,
  0.8450000286102295,
  0.8424999713897705,
  0.9300000071525574,
  0.9225000143051147,
  0.8224999904632568],
 'mean': [0.5456249862909317,
  0.6050000041723251,
  0.6068750023841858,
  0.809374988079071,
  0.6724999994039536,
  0.5487499982118607,
  0.7981249988079071,
  0.7406249940395355,
  0.8668750077486038,
  0.8656250089406967,
  0.8681250065565109,
  0.8693749904632568,
  0.9474999904632568,
  0.9506250023841858,
  0.8599999994039536],
 'max': [0.5799999833106995,
  0.6800000071525574,
  0.6575000286102295,
  0.8274999856948853,
  0.6949999928474426,
  0.6299999952316284,
  0.8349999785423279,
  0.7549999952316284,
  0.8824999928474426,
  0.875,
  0.8774999976158142,
  0.8924999833106995,
  0.9574999809265137,
  0.9700000286102295,
  0.8849999904632568]}

In [ ]:
fence_curve = {'min': [0.6666666865348816,
  0.21875,
  0.4166666567325592,
  0.5208333134651184,
  0.25,
  0.2395833283662796,
  0.6354166865348816,
  0.46875,
  0.625,
  0.6354166865348816,
  0.6666666865348816,
  0.6770833134651184,
  0.9895833134651184,
  0.7083333134651184,
  0.65625],
 'mean': [0.6927083283662796,
  0.4531249925494194,
  0.4921875,
  0.5442708283662796,
  0.3307291716337204,
  0.3619791604578495,
  0.7265625149011612,
  0.5833333283662796,
  0.6640625149011612,
  0.6510416567325592,
  0.7187500149011612,
  0.703125,
  0.9973958283662796,
  0.9192708283662796,
  0.6874999850988388],
 'max': [0.71875,
  0.8020833134651184,
  0.5625,
  0.5729166865348816,
  0.5104166865348816,
  0.5625,
  0.7916666865348816,
  0.7708333134651184,
  0.6979166865348816,
  0.6770833134651184,
  0.78125,
  0.7604166865348816,
  1.0,
  1.0,
  0.7083333134651184]}

In [ ]:
hard_curve = {'min': [0.6145833134651184,
  0.1979166716337204,
  0.3333333432674408,
  0.2708333432674408,
  0.3333333432674408,
  0.3020833432674408,
  0.4583333432674408,
  0.46875,
  0.75,
  0.7083333134651184,
  0.7708333134651184,
  0.6145833134651184,
  0.9270833134651184,
  0.9583333134651184,
  0.8229166865348816],
 'mean': [0.6822916716337204,
  0.3489583320915699,
  0.6223958432674408,
  0.6744791641831398,
  0.5078124925494194,
  0.4062499925494194,
  0.643229179084301,
  0.5338541716337204,
  0.7994791716337204,
  0.7473958432674408,
  0.8020833283662796,
  0.7161458283662796,
  0.9609375,
  0.9713541716337204,
  0.8307291716337204],
 'max': [0.7604166865348816,
  0.5,
  0.8541666865348816,
  0.875,
  0.6145833134651184,
  0.5208333134651184,
  0.7604166865348816,
  0.59375,
  0.8541666865348816,
  0.8229166865348816,
  0.8333333134651184,
  0.7916666865348816,
  0.9791666865348816,
  0.9791666865348816,
  0.84375]}

In [ ]:
easy_curve = {'min': [0.7916666865348816,
  0.4583333432674408,
  0.4375,
  0.90625,
  0.7291666865348816,
  0.5,
  0.96875,
  0.9375,
  0.9166666865348816,
  0.8958333134651184,
  0.90625,
  0.8958333134651184,
  0.96875,
  0.9375,
  0.8958333134651184],
 'mean': [0.8098958432674408,
  0.6953125074505806,
  0.5364583358168602,
  0.9479166567325592,
  0.8177083283662796,
  0.65625,
  0.984375,
  0.9661458432674408,
  0.9322916716337204,
  0.9192708283662796,
  0.9296875,
  0.9244791567325592,
  0.9791666567325592,
  0.9557291716337204,
  0.9010416567325592],
 'max': [0.84375,
  0.90625,
  0.78125,
  0.9895833134651184,
  0.96875,
  0.8229166865348816,
  1.0,
  1.0,
  0.9479166865348816,
  0.9375,
  0.9583333134651184,
  0.9583333134651184,
  0.9895833134651184,
  0.9791666865348816,
  0.90625]}

In [ ]:
guard_curve = {'min': [0.6354166865348816,
  0.4895833432674408,
  0.4791666567325592,
  0.8333333134651184,
  0.7708333134651184,
  0.4895833432674408,
  0.9791666865348816,
  0.8645833134651184,
  0.9166666865348816,
  0.9375,
  0.9270833134651184,
  0.9583333134651184,
  0.96875,
  0.96875,
  0.9270833134651184],
 'mean': [0.6927083283662796,
  0.690104179084301,
  0.5703124925494194,
  0.8828124850988388,
  0.8229166567325592,
  0.6562500074505806,
  0.9947916716337204,
  0.9609375,
  0.9427083432674408,
  0.9609375,
  0.9401041716337204,
  0.9817708283662796,
  0.9765625,
  0.9843749850988388,
  0.9374999850988388],
 'max': [0.75,
  0.9166666865348816,
  0.8333333134651184,
  0.9270833134651184,
  0.8958333134651184,
  0.8125,
  1.0,
  1.0,
  0.9791666865348816,
  0.9895833134651184,
  0.9479166865348816,
  1.0,
  0.9895833134651184,
  0.9895833134651184,
  0.96875]}